<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%204/Modulo_4_Ejercicio_4_Optimizaci%C3%B3n_M%C3%A9tricas_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Ejercicio 4: Optimización de Métricas y Selección de Modelos (Nivel: Avanzado)**
#### Optimización de un modelo de clasificación multiclase para defectos en productos electrónicos

### **Introducción**
En este ejercicio, exploraremos cómo optimizar un modelo de clasificación multiclase y seleccionar la mejor métrica de evaluación.

El dataset contiene información sobre defectos en productos electrónicos, con 4 categorías de defecto. Aplicaremos técnicas como **SMOTE para balanceo de clases**, **selección de características**, y **Random Forest** para mejorar la precisión en la clasificación. Evaluaremos el modelo con **Precisión, Recall, F1 Score, Exactitud** y compararemos **F1 Score Macro vs Micro y la Curva ROC para clasificación multiclase**.

In [ ]:
# Celda 1: Carga de librerías y configuración del entorno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize

# Configuración de gráficos
sns.set(style='whitegrid')

In [ ]:
# Celda 2: Carga del dataset
dataset_path = 'https://github.com/dtoralg/IE_Calidad_ML/raw/main/Data/defectos_productos.csv'
...

# Mostrar las primeras filas del dataset
...

In [ ]:
# Celda 3: Información general del dataset
...

In [ ]:
# Celda 4: Distribución de la variable objetivo
...

### **Preprocesamiento de los datos**
Se realizará un proceso de preparación de datos que incluye:
- **Balanceo de clases con SMOTE** para mejorar la detección de todas las clases.
- **Selección de características** para eliminar variables no relevantes.
- **Codificación de variables categóricas y normalización de variables numéricas**.

In [ ]:
# Celda 5: Procesamiento de datos
# Eliminar duplicados
...

# Imputar valores faltantes con la mediana para variables numéricas
...
# Codificar variables categóricas con OneHotEncoder
...

# Codificar la variable objetivo
...

# División en variables predictoras y objetivo
...

# División en entrenamiento y prueba
...

# Aplicación de SMOTE para balancear las clases
...

# Normalización de los datos
...

In [ ]:
# Celda 6: Entrenamiento del modelo con Random Forest
modelo = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

In [ ]:
# Celda 7: Evaluación del modelo con classification report
# Importante, usa el argumento "target_names" para especificar las clases
...

In [ ]:
# Celda 8: Visualización de resultados

# Matriz de Confusión
...

In [ ]:
# Comparación entre clases reales y predichas
...

In [ ]:
# Curva ROC Multiclase
...

plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC para Clasificación Multiclase')
plt.legend()
plt.show()

### **Conclusiones**
- Se implementó **SMOTE** para mejorar la detección de clases minoritarias y equilibrar el dataset.
- **Random Forest** mostró una mejora significativa en comparación con un Árbol de Decisión simple, gracias a su capacidad de generalización y manejo de datos no lineales.
- **El F1 Score Macro vs Micro** permite entender cómo el modelo trata clases balanceadas y desbalanceadas. El **F1 Macro** evalúa el rendimiento en todas las clases por igual, mientras que el **F1 Micro** pondera según la cantidad de muestras en cada clase.
- **La Curva ROC-AUC multiclase** permitió analizar la capacidad del modelo para distinguir correctamente entre diferentes defectos.

### **Posibles Mejoras**
- **Ajuste de hiperparámetros**: Se puede optimizar el modelo utilizando **GridSearchCV** para encontrar los mejores valores de `n_estimators`, `max_depth`, y otros parámetros.
- **Exploración de otros modelos**: Probar modelos más avanzados como **XGBoost o LightGBM**, que pueden ofrecer mejores resultados en tareas de clasificación complejas.
- **Ingeniería de características**: Crear nuevas variables derivadas de las existentes para mejorar la capacidad predictiva del modelo.
- **Manejo avanzado de datos desbalanceados**: Probar otras técnicas como **BalancedRandomForestClassifier** o **focal loss** para mejorar la detección de defectos minoritarios.
- **Reducción de dimensionalidad**: Aplicar **PCA** para evaluar si la reducción de dimensiones mejora la eficiencia del modelo sin afectar significativamente el rendimiento.
